## <font color='#0000FF'>MSC_DA_CA2<font color='#1ABC9D'>
### <font color='#'>**Advanced Data Analytics  & Big Data Storage and Processing**
### <font color='#1ABC9C'>**Lecturer(s): David McQuaid and Muhammad Iqbal**
------
<font color='#1ABC9C'>**Students name / ID** // Rosilene Francisca da Silva - 2021090

In [1]:
#Importing the libraries.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib import cm
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

/home/hduser/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#Command to display all columns in the file.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Importing the Dataset 
tweets_df = pd.read_csv('ProjectTweets.csv')
tweets_df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,5,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [4]:
tweets_df.shape

(1599999, 6)

In [5]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column                                                                                                               Non-Null Count    Dtype 
---  ------                                                                                                               --------------    ----- 
 0   0                                                                                                                    1599999 non-null  int64 
 1   1467810369                                                                                                           1599999 non-null  int64 
 2   Mon Apr 06 22:19:45 PDT 2009                                                                                         1599999 non-null  object
 3   NO_QUERY                                                                                                             1599999 non-null  object
 4   _

Based on the first basic details, the dataset appears to be a collection of tweets, commonly can be use in sentiment analysis or other natural language processing tasks. 

Dataset Size and Integrity:
The dataset contains approximately 1.6 million entries (1599999 entries, indexed from 0 to 1599998).
Each column has the same number of non-null entries (1599999), indicating there are no missing values in any of the columns.
Memory Usage: The dataframe is consuming approximately 73.2 MB of memory, which is relevant for understanding the computation resources needed to process this dataset.

In [6]:
tweets_df.dtypes

0                                                                                                                       int64
1467810369                                                                                                              int64
Mon Apr 06 22:19:45 PDT 2009                                                                                           object
NO_QUERY                                                                                                               object
_TheSpecialOne_                                                                                                        object
@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D    object
dtype: object

In [7]:
tweets_df.isnull().sum()

0                                                                                                                      0
1467810369                                                                                                             0
Mon Apr 06 22:19:45 PDT 2009                                                                                           0
NO_QUERY                                                                                                               0
_TheSpecialOne_                                                                                                        0
@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D    0
dtype: int64

In [8]:
tweets_df.isnull().values.any()

False

In [9]:
tweets_df.rename(columns={"0": "index","1467810369": "tweet_id",
    "Mon Apr 06 22:19:45 PDT 2009": "date","NO_QUERY": "query_status",
    "_TheSpecialOne_": "user_handle",
    "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D":
                          "tweet_text"}, inplace=True)

In [10]:
tweets_df.head()

,index,tweet_id,date,query_status,user_handle,tweet_text
0,1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,5,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [11]:
tweets_df.query_status.value_counts()

query_status
NO_QUERY    1599999
Name: count, dtype: int64

The feature `query_status` shows whether any specific query key phrase has been used to collect the tweets; nonetheless, 100% of the entries in this column have the value "NO_QUERY."

In [12]:
#### Display the Unique Values

In [13]:
for column in tweets_df.columns:
    unique_values = tweets_df[column].unique()
    print(f"Unique values in '{column}': {unique_values}")

Unique values in 'index': [      1       2       3 ... 1599997 1599998 1599999]
Unique values in 'tweet_id': [1467810672 1467810917 1467811184 ... 2193601991 2193602064 2193602129]
Unique values in 'date': ['Mon Apr 06 22:19:49 PDT 2009' 'Mon Apr 06 22:19:53 PDT 2009'
 'Mon Apr 06 22:19:57 PDT 2009' ... 'Tue Jun 16 08:38:58 PDT 2009'
 'Tue Jun 16 08:39:00 PDT 2009' 'Tue Jun 16 08:40:50 PDT 2009']
Unique values in 'query_status': ['NO_QUERY']
Unique values in 'user_handle': ['scotthamilton' 'mattycus' 'ElleCTF' ... 'EvolveTom' 'AmandaMarie1028'
 'bpbabe']
Unique values in 'tweet_text': ["is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"
 '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds'
 'my whole body feels itchy and like its on fire ' ...
 'Are you ready for your MoJo Makeover? Ask me for details '
 'Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur '
 'happy #charit

In [14]:
def return_unique_values(data_frame):
    unique_dataframe = pd.DataFrame()
    unique_dataframe['Features'] = data_frame.columns
    uniques = []
    for col in data_frame.columns:
        u = data_frame[col].nunique()
        uniques.append(u)
    unique_dataframe['Uniques'] = uniques
    return unique_dataframe

unidf = return_unique_values(tweets_df)
print(unidf)

       Features  Uniques
0         index  1599999
1      tweet_id  1598314
2          date   774362
3  query_status        1
4   user_handle   659775
5    tweet_text  1581465


In [15]:
tweets_df.describe()

,index,tweet_id
count,1.599999e+06,1.599999e+06
mean,8.000000e+05,1.998818e+09
std,4.618801e+05,1.935757e+08
min,1.000000e+00,1.467811e+09
25%,4.000005e+05,1.956916e+09
50%,8.000000e+05,2.002102e+09
75%,1.200000e+06,2.177059e+09
max,1.599999e+06,2.329206e+09


#### To begin with, dropping the columns that are unnecessary for the particular goal of sentiment analysis.The `index`, `query_status`,`tweet_id`,columns in this instance.